In [ ]:
# PREGUNTES PER AL THOMAS
# 1. Pitch interpolated and smoothed? (The first row returns NaN)
# 2. How to get Melodia working?
# 3. 
# 4.
'''
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install IPython
'''
!pip install librosa

## 0. Imports

In [ ]:

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio

In [ ]:
!pip install gdown
!pip install configobj

In [ ]:
#%pip install compiam

## 1. Load files

### 1.1 Annotations

In [ ]:
annotations_path = "../data/raw/annotations_koti_janmani.txt"

In [ ]:
def to_seconds(t):
    return (t.hour * 60 * 60) + (t.minute * 60) + t.second + (t.microsecond / 1000000)

def load_annotations_file(path: str) -> pd.DataFrame:
    """
    Load annotations from a file.

    :param path: Path to the file containing the annotations.
    :return: A pandas DataFrame containing the annotations.
    """
    # Read the annotations file
    annotations = pd.read_csv(path, sep='\t', header=None)

    # Add column names
    annotations.columns = ["level", "", "start", "end", "duration", "label"]
    del annotations[""]

    # Convert to seconds
    annotations["start"] = pd.to_datetime(annotations["start"])
    annotations["end"] = pd.to_datetime(annotations["end"])
    annotations["start"] = annotations["start"].apply(to_seconds)
    annotations["end"] = annotations["end"].apply(to_seconds)

    annotations.reset_index(inplace=True)

    return annotations

In [ ]:
annotations_kj = load_annotations_file(annotations_path)
annotations_kj

In [ ]:
annotations_sancara = annotations_kj[annotations_kj["level"] == "sancara"]
annotations_sancara

### 1.2 Audio

In [ ]:
audio_path = "../data/raw/Koti Janmani/Koti Janmani.multitrack-vocal.mp3"
#audio_path = "../data/raw/Vanajaksha Ninne Kori/Vanajaksha Ninne Kori_vocal.mp3"

In [ ]:
def load_audio_file(path: str, sampling_rate: int) -> tuple:
    audio_time_series, sr = librosa.load(path, sr=sampling_rate)
    return audio_time_series, sr

In [ ]:
audio_kj, sr_kj = load_audio_file(audio_path, 44100)
Audio(data=audio_kj, rate=sr_kj)

### 1.3 Extract pitch

In [ ]:
# Passar la ref del "Koti Janmani.ctonic.txt" a una constant (per fer el canvi de Hz a cents)
tonic_path = "../data/raw/Koti Janmani/Koti Janmani.ctonic.txt"

with open(tonic_path, "r") as f:
    ctonic_ref = float(f.readline().strip())

ctonic_ref

In [ ]:
from scipy.signal import savgol_filter

def pitch_to_cents(pitch: float, ref: float):
    if pitch == 0:
        return None
    else:
        return 1200 * math.log(pitch/ref, 2)

def interpolate_and_smooth_pitch(pitch):
    pitch = pd.Series(pitch)
    pitch[pitch <= 0] = np.nan
    pitch_interpolated = pitch.interpolate(method="linear")
    pitch_smoothed = savgol_filter(pitch_interpolated, window_length=5, polyorder=2)
    return pitch_smoothed

#### Extract pitch from pitch file

In [ ]:
pitch_path_kj = "../data/raw/Koti Janmani/Koti Janmani.pitch.txt"

In [ ]:
def load_pitch_file(path: str):
    """
    Load a pitch file from a given path.

    :param path: Path to the pitch file.
    :return: pitch_file, time, pitch, timestep
    """
    pitch_file = pd.read_csv(path, sep="\t", header=None)
    pitch_file.columns = ["time", "pitch"]

    time = pitch_file["time"].values
    pitch = pitch_file["pitch"].values
    timestep = time[1] - time[0]

    return pitch_file, time, pitch, timestep


In [ ]:
pitch_file_kj, time_kj, pitch_kj, timestep_kj = load_pitch_file(pitch_path_kj)

# Replace non-positive values with NaN, interpolate and smooth
pitch_kj_smoothed = interpolate_and_smooth_pitch(pitch_kj)

# Convert pitch to cents (TODO: should we use the smoothed? why?)
pitch_cents_kj = np.array([pitch_to_cents(p, ctonic_ref) for p in pitch_kj_smoothed])

In [ ]:
#pitch_file_kj
pitch_cents_kj[10000:10020]

#### NOT WORKING YET (extracting pitch from audio file using Melodia)

In [ ]:
import importlib.util
if importlib.util.find_spec('compiam') is None:
    ## Bear in mind this will only run in a jupyter notebook / Collab session
    %pip install compiam

import compiam
#!git clone https://github.com/MTG/essentia.git


In [ ]:
%pip install wheel


In [ ]:
#%pip install essentia --use-pep517
%pip install essentia --no-cache-dir

In [ ]:
#import essentia
# Extract pitch using Melodia
import essentia
#import essentia/src/algorithms/standard as es
from compiam.melody.pitch_extraction import Melodia
from compiam.melody.pitch_extraction import FTANetCarnatic
# from compiam.melody.pitch_extraction import FTANetCarnatic


melodia = Melodia()
# ftanet = FTANetCarnatic()

melodia_pitch_track = melodia.extract(audio_path)

# Separate pitch and time
pitch = melodia_pitch_track[:, 1]
time = melodia_pitch_track[:, 0]

timestep = time[1] - time[0]

# Convert pitch to cents
pitch_cents = np.array([pitch_to_cents(p, ctonic_ref) for p in pitch])


#save melodia pitch track into csv file in same folder
melodia_pitch_track_df = pd.DataFrame(melodia_pitch_track, columns=["time", "pitch"])



In [ ]:
#melodia_pitch_track_df.to_csv("../data/raw/Koti Janmani/Koti Janmani.melodia.pitch.txt", sep="\t", index=False)

#save to csv without creating a dataframe first row time second pitch
np.savetxt("../data/raw/Koti Janmani/Koti Janmani.melodia.pitch.txt", melodia_pitch_track, delimiter="\t")
#np.savetxt("../data/raw/Vanajaksha Ninne Kori/Vanajaksha Ninne Kori.melodia.pitch.txt", melodia_pitch_track, delimiter="\t")

#melodia_pitch_track_df.to_csv("../data/raw/Vanajaksha Ninne Kori/Vanajaksha Ninne Kori.melodia.pitch.txt", sep="\t", index=False)


In [ ]:
# CODE EXAMPLE to extract some features from an annotation

# Start, end points, and label of pattern from annotations
s1 = 5.238
s2 = 8.300
label = 'mgmpmmgrg'

# convert to number of sequence elements
s1_ix = round(s1/timestep)
s2_ix = round(s2/timestep)

# Segment pitch curve to correspond only to that pattern
samp_time = time[s1_ix:s2_ix]
samp_pitch = pitch_cents[s1_ix:s2_ix]

# Plot pattern
plt.plot(samp_time, samp_pitch)
plt.title(label)
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')

# Remove none values corresponding to silence
samp_pitch_clean = [x for x in samp_pitch if x != None]

# Extract features from cleaned cents sample...
max(samp_pitch_clean)
min(samp_pitch_clean)

## 2. Feature extraction

### 2.1 Time Domain Features

In [ ]:
from librosa.feature import rms
from librosa.feature import zero_crossing_rate as zcr

In [ ]:
time_features_df = annotations_sancara.copy()

#### 2.1.1 Amplitude Envelope ¿¿??

In [ ]:
# No se si serveix d'algo

#### 2.1.2 Root Mean Square Energy

In [ ]:
def compute_rms(audio: np.ndarray, sample_start: float, sample_end: float, sr: int) -> float:
    sample = audio[round(sample_start * sr):round(sample_end * sr)]
    return np.mean(rms(y=sample)[0])

In [ ]:
# NOTE: a stands for annotation
time_features_df['rmse'] = time_features_df.apply(
    lambda a: compute_rms(audio_kj, a['start'], a['end'], sr_kj), axis=1
)

#### 2.1.3 Zero Crossing Rate

In [ ]:
def compute_zcr(audio: np.ndarray, sample_start: float, sample_end: float, sr: int) -> float:
    sample = audio[round(sample_start * sr):round(sample_end * sr)]
    return np.mean(zcr(y=sample)[0])

In [ ]:
time_features_df['zcr'] = time_features_df.apply(
    lambda a: compute_zcr(audio_kj, a['start'], a['end'], sr_kj), axis=1
)

#### Time Domain Features DataFrame

In [ ]:
time_features_df

### 2.2 Frequency Domain Features

In [ ]:
from librosa.feature import spectral_centroid as scentroid
from librosa.feature import spectral_bandwidth as sbandwidth
from librosa.feature import spectral_rolloff as srolloff
from librosa.feature import mfcc

In [ ]:
frequency_features_df = annotations_sancara.copy()

#### 2.2.1 Band Energy Ratio (not necessary??)

In [ ]:
# TODO: Passar el codi de l'altre notebook

#### 2.2.2 Spectral Centroid

Each frame of a magnitude spectrogram is normalized and treated as a distribution over frequency bins, from which the mean (centroid) is extracted per frame.

In [ ]:
def compute_scentroid(audio: np.ndarray, sample_start: float, sample_end: float, sr: int) -> float:
    sample = audio[round(sample_start * sr):round(sample_end * sr)]
    return np.mean(scentroid(y=sample, sr=sr)[0])

In [ ]:
frequency_features_df['spectral_centroid'] = frequency_features_df.apply(
    lambda a: compute_scentroid(audio_kj, a['start'], a['end'], sr_kj), axis=1
)

#### 2.2.3 Spectral Bandwidth

In [ ]:
def compute_sbandwidth(audio: np.ndarray, sample_start: float, sample_end: float, sr: int) -> float:
    sample = audio[round(sample_start * sr):round(sample_end * sr)]
    return np.mean(sbandwidth(y=sample, sr=sr)[0])

In [ ]:
frequency_features_df['spectral_bandwidth'] = frequency_features_df.apply(
    lambda a: compute_sbandwidth(audio_kj, a['start'], a['end'], sr_kj), axis=1
)

#### 2.2.4 Spectral Rolloff

The roll-off frequency is defined for each frame as the center frequency for a spectrogram bin such that at least roll_percent (0.85 by default) of the energy of the spectrum in this frame is contained in this bin and the bins below. This can be used to, e.g., approximate the maximum (or minimum) frequency by setting roll_percent to a value close to 1 (or 0).

In [ ]:
def compute_srolloff(audio: np.ndarray, sample_start: float, sample_end: float, sr: int) -> float:
    sample = audio[round(sample_start * sr):round(sample_end * sr)]
    return np.mean(srolloff(y=sample, sr=sr)[0])

In [ ]:
frequency_features_df['spectral_rolloff'] = frequency_features_df.apply(
    lambda a: compute_srolloff(audio_kj, a['start'], a['end'], sr_kj), axis=1
)

#### 2.2.5 Mel Frequency Cepstral Coefficients

In [ ]:
mfcc_df = annotations_sancara.copy()

In [ ]:
def compute_mfcc(audio: np.ndarray, sample_start: float, sample_end: float, sr: int) -> np.ndarray:
    sample = audio[round(sample_start * sr):round(sample_end * sr)]
    return np.mean(mfcc(y=sample, sr=sr, n_mfcc=13), axis=1)

mfcc_cols = [f'mfcc_{i+1}' for i in range(13)]

In [ ]:
mfcc_df[mfcc_cols] = mfcc_df.apply(
    lambda a: compute_mfcc(audio_kj, a['start'], a['end'], sr_kj), axis=1
).apply(pd.Series)

#### Frequency Domain Features DataFrame

In [ ]:
#frequency_features_df
#mfcc_df

#### 2.2.4 Spectral Contrast ¿¿??

Each frame of a spectrogram S is divided into sub-bands. For each sub-band, the energy contrast is estimated by comparing the mean energy in the top quantile (peak energy) to that of the bottom quantile (valley energy). High contrast values generally correspond to clear, narrow-band signals, while low contrast values correspond to broad-band noise.

### 2.3 Pitch Curve Features

In [ ]:
pitch_features_df = annotations_sancara.copy()

#### 2.3.1 Mean pitch, Min/Max and Range

In [ ]:
# TODO: Not sure if the min_pitch and max_pitch correspond to each annotation

def get_mean_min_max_pitch(cents: np.ndarray, tstep: float, sample_start: float, sample_end: float):
    #sample_time = time[round(sample_start/tstep):round(sample_end/tstep)]
    sample_cents = cents[round(sample_start/tstep):round(sample_end/tstep)]
    """ sample_cents_clean = [x for x in sample_cents if x is not None]
    if not sample_cents_clean:
        return None """
    return np.mean(sample_cents), min(sample_cents), max(sample_cents)

In [ ]:
pitch_features_df[['mean_pitch', 'min_pitch', 'max_pitch']] = pitch_features_df.apply(
    lambda a: get_mean_min_max_pitch(pitch_cents_kj, timestep_kj, a['start'], a['end']), axis=1
).apply(pd.Series)

In [ ]:
# Range
pitch_features_df['pitch_range'] = pitch_features_df['max_pitch'] - pitch_features_df['min_pitch']

#### 2.3.2 Number of Change Points

In [ ]:
from scipy.signal import find_peaks

def compute_number_of_change_points(cents: np.ndarray, tstep: float, sample_start: float, sample_end: float) -> int:
    #sample_time = time[round(sample_start/tstep):round(sample_end/tstep)]
    sample_cents = cents[round(sample_start/tstep):round(sample_end/tstep)]

    peaks, _ = find_peaks(sample_cents)
    valleys, _ = find_peaks(-sample_cents)

    num_change_points = len(peaks) + len(valleys)
    return num_change_points

In [ ]:
pitch_features_df['num_change_points'] = pitch_features_df.apply(
    lambda a: compute_number_of_change_points(pitch_cents_kj, timestep_kj, a['start'], a['end']), axis=1
)

#### Pitch Curve Features DataFrame

In [ ]:
pitch_features_df

### 2.4 Create DataFrame with the Features

In [ ]:
cols_to_drop = ["index", "level", "start", "end", "duration","label"]
features_df = pd.concat([annotations_sancara, 
                        time_features_df.drop(columns=cols_to_drop),
                        frequency_features_df.drop(columns=cols_to_drop),
                        mfcc_df.drop(columns=cols_to_drop),
                        pitch_features_df.drop(columns=cols_to_drop)],
axis=1)

# TODO: Normalize the data
# ...

In [ ]:
features_df

## 3. Modelling

In [ ]:
df = annotations_kj.copy()

In [ ]:
df = pd.get_dummies(df, columns=['label'])
df

### 3.1 Get Training Data

In [ ]:
import sklearn

from sklearn.model_selection import train_test_split

In [ ]:
# Falten BER.
features = ['rmse', 'zcr', 
            'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff',
            'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 
            'mean_pitch', 'min_pitch', 'max_pitch', 'pitch_range', 'num_change_points']
target = ['label'] # ¿?

In [ ]:
X = df[features].values
y = df[target].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
len(X_train)

### 3.2 Train a Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(...)

In [ ]:
clf.fit(X_train, y_train)

## 4. Evaluation